# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I5, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [2]:
a = rand(10^7); # 1D vector of random numbers, uniform on [0,1)

In [3]:
sum(a)

5.000643397143609e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [4]:
@time sum(a)

  0.004108 seconds (1 allocation: 16 bytes)


5.000643397143609e6

In [5]:
@time sum(a)

  0.004742 seconds (1 allocation: 16 bytes)


5.000643397143609e6

In [6]:
@time sum(a)

  0.004289 seconds (1 allocation: 16 bytes)


5.000643397143609e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [7]:
using Pkg
Pkg.add("BenchmarkTools")

   Updating registry at `C:\Users\Victor\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [======>                                  ]  14.1 %

Fetching: [========================================>]  1009 %Fetching: [==============>                          ]  33.8 %>                   ]  50.6 %]  55.7 %.2 %           ]  72.3 %Fetching: [===================================>     ]  86.0 %4 %.0 %

  Resolving package versions...
  Installed Zlib_jll ─────── v1.2.11+11
  Installed OpenSSL_jll ──── v1.1.1+3
  Installed BenchmarkTools ─ v0.5.0
   Updating `C:\Users\Victor\.julia\environments\v1.4\Project.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0
   Updating `C:\Users\Victor\.julia\environments\v1.4\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0
  [458c3c95] ↑ OpenSSL_jll v1.1.1+2 ⇒ v1.1.1+3
  [83775a58] ↑ Zlib_jll v1.2.11+10 ⇒ v1.2.11+11


In [8]:
using BenchmarkTools  

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [10]:
Pkg.add("PackageCompiler")


  Resolving package versions...
  Installed PackageCompiler ─ v1.1.1
   Updating `C:\Users\Victor\.julia\environments\v1.4\Project.toml`
  [9b87118b] + PackageCompiler v1.1.1
   Updating `C:\Users\Victor\.julia\environments\v1.4\Manifest.toml`
  [9b87118b] + PackageCompiler v1.1.1


In [33]:
using PackageCompiler

┌ Info: Precompiling PackageCompiler [9b87118b-4619-50d2-8e1e-99f35a4d4d9d]
└ @ Base loading.jl:1260


In [34]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -o 'C:\Users\Victor\AppData\Local\Temp\jl_6iDy18PrUJ.dll' -`: no such file or directory (ENOENT)

In [ ]:
c_sum(a)

In [ ]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

In [ ]:
c_sum(a) - sum(a)  

In [ ]:
≈  # alias for the `isapprox` function

In [ ]:
?isapprox

We can now benchmark the C code directly from Julia:

In [ ]:
c_bench = @benchmark c_sum($a)

In [ ]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

In [21]:
d = Dict()  # a "dictionary", i.e. an associative array

Dict{Any,Any} with 0 entries

In [ ]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

In [ ]:
using Plots
gr()

In [ ]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
        xlim=(m - 0.01, m + σ),
        xlabel="milliseconds", ylabel="count", label="")

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [ ]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

In [ ]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [31]:
using Pkg; Pkg.add("PyCall")
using PyCall

  Resolving package versions...
   Updating `C:\Users\Victor\.julia\environments\v1.4\Project.toml`
  [438e738f] + PyCall v1.91.4
   Updating `C:\Users\Victor\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [32]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=atol>0 ? 0 : √eps, atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significand digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to the usual `norm` function in LinearAlgebra, but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

# Examples

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true
```


In [35]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [36]:
pysum(a)

5.000643397144246e6

In [39]:
$a

ErrorException: syntax: "$" expression outside quote

In [37]:
pysum(a) ≈ sum(a)

true

In [38]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     1.318 s (0.00% GC)
  median time:      1.338 s (0.00% GC)
  mean time:        1.339 s (0.00% GC)
  maximum time:     1.361 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [40]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "Julia hand-written simd" => 3.6455
  "Julia hand-written"      => 10.9088
  "Julia built-in"          => 3.6307
  "Python built-in"         => 1318.4

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [41]:
using Pkg; Pkg.add("Conda")
using Conda

  Resolving package versions...
   Updating `C:\Users\Victor\.julia\environments\v1.4\Project.toml`
  [8f4d0f93] + Conda v1.4.1
   Updating `C:\Users\Victor\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [42]:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda C:\Users\Victor\.julia\packages\Conda\3rPhK\src\Conda.jl:113


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Victor\.julia\conda\3

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.2         |           py37_0         157 KB
    ------------------------------------------------------------
                                           Total:         157 KB

The following packages will be UPDATED:

  certifi                                 2020.4.5.1-py37_0 --> 2020.4.5.2-py37_0



certifi-2020.4.5.2   | 157 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [43]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at In[43]:1
└ @ Core In[43]:1


BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     9.155 ms (0.00% GC)
  median time:      9.611 ms (0.00% GC)
  mean time:        9.647 ms (0.00% GC)
  maximum time:     13.033 ms (0.00% GC)
  --------------
  samples:          518
  evals/sample:     1

In [44]:
numpy_sum(a)

5.00064339714361e6

In [45]:
numpy_sum(a) ≈ sum(a)

true

In [46]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "Julia hand-written simd" => 3.6455
  "Python numpy"            => 9.1554
  "Julia hand-written"      => 10.9088
  "Julia built-in"          => 3.6307
  "Python built-in"         => 1318.4

# 5. Python, hand-written 

In [47]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x00000000526D15E8>

In [48]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     1.705 s (0.00% GC)
  median time:      1.713 s (0.00% GC)
  mean time:        1.714 s (0.00% GC)
  maximum time:     1.723 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

In [49]:
sum_py(a)

5.000643397144246e6

In [50]:
sum_py(a) ≈ sum(a)

true

In [51]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "Julia hand-written simd" => 3.6455
  "Python numpy"            => 9.1554
  "Julia hand-written"      => 10.9088
  "Python hand-written"     => 1705.15
  "Julia built-in"          => 3.6307
  "Python built-in"         => 1318.4

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [18]:
@which sum(a)

sum(a::AbstractArray; dims) in Base at reducedim.jl:652

In [19]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.631 ms (0.00% GC)
  median time:      3.746 ms (0.00% GC)
  mean time:        3.799 ms (0.00% GC)
  maximum time:     6.034 ms (0.00% GC)
  --------------
  samples:          1314
  evals/sample:     1

In [22]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 3.6307

# 7. Julia (hand-written) 

In [23]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [24]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.909 ms (0.00% GC)
  median time:      10.947 ms (0.00% GC)
  mean time:        11.055 ms (0.00% GC)
  maximum time:     13.556 ms (0.00% GC)
  --------------
  samples:          452
  evals/sample:     1

In [25]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Julia hand-written" => 10.9088
  "Julia built-in"     => 3.6307

# 8. Julia (hand-written w. simd) 

In [26]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [27]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.646 ms (0.00% GC)
  median time:      3.799 ms (0.00% GC)
  mean time:        3.853 ms (0.00% GC)
  maximum time:     10.057 ms (0.00% GC)
  --------------
  samples:          1296
  evals/sample:     1

In [28]:
mysum_simd(a)

5.000643397143612e6

In [29]:
mysum_simd(a) ≈ sum(a)

true

In [30]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "Julia hand-written simd" => 3.6455
  "Julia hand-written"      => 10.9088
  "Julia built-in"          => 3.6307

# Summary

In [52]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............3.6
Julia hand-written simd.....3.6
Python numpy................9.2
Julia hand-written.........10.9
Python built-in..........1318.4
Python hand-written......1705.1
